In [2]:
#!pip install kaggle
#!pip install pandas numpy sklearn

## Imports e Especificações de Libs

In [3]:
import pandas as pd 
import numpy as np 
import difflib
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

## EDA

In [4]:
path_to_data = r"data/"

In [5]:
user_review = pd.read_csv(f'{path_to_data}animelist.csv')
anime_list = pd.read_csv(f'{path_to_data}anime.csv')
anime_description = pd.read_csv(f'{path_to_data}anime_with_synopsis.csv')

In [6]:
#How many users
len(user_review.user_id.unique())

325770

In [7]:
#Showing first 10 anime ratings
'''for index, row in user_review.iterrows():
    print(index,row)
    if(index > 10):
        break'''

'for index, row in user_review.iterrows():\n    print(index,row)\n    if(index > 10):\n        break'

In [8]:
anime_full = pd.merge(anime_list, anime_description[["MAL_ID","sypnopsis"]], how='inner', on="MAL_ID")

In [9]:
anime_description.columns

Index(['MAL_ID', 'Name', 'Score', 'Genres', 'sypnopsis'], dtype='object')

Missing Data

In [10]:
anime_full.isnull().sum().tail(5)

Score-4      0
Score-3      0
Score-2      0
Score-1      0
sypnopsis    8
dtype: int64

Choosing Features

In [11]:
anime_full.columns

Index(['MAL_ID', 'Name', 'Score', 'Genres', 'English name', 'Japanese name',
       'Type', 'Episodes', 'Aired', 'Premiered', 'Producers', 'Licensors',
       'Studios', 'Source', 'Duration', 'Rating', 'Ranked', 'Popularity',
       'Members', 'Favorites', 'Watching', 'Completed', 'On-Hold', 'Dropped',
       'Plan to Watch', 'Score-10', 'Score-9', 'Score-8', 'Score-7', 'Score-6',
       'Score-5', 'Score-4', 'Score-3', 'Score-2', 'Score-1', 'sypnopsis'],
      dtype='object')

In [12]:
anime_full.sample(1)

,MAL_ID,Name,Score,Genres,English name,Japanese name,Type,Episodes,Aired,Premiered,...,Score-9,Score-8,Score-7,Score-6,Score-5,Score-4,Score-3,Score-2,Score-1,sypnopsis
3851,6184,Makaryuudo Demon Hunter,5.41,"Supernatural, Demons",Unknown,魔狩人,OVA,1,"Nov 8, 1989",Unknown,...,5.0,39.0,105.0,190.0,181.0,99.0,59.0,30.0,12.0,Yama Rikudo is not your average teenage girl. ...


In [13]:
features = ["Name", "Genres", "Episodes","Studios","Rating","Score","Aired","sypnopsis"]

In [14]:
anime_full.sypnopsis.fillna(' ', inplace=True)
anime_full.isna().sum().sum()

0

In [15]:
featured = anime_full[features].apply(lambda row: '_'.join(row.values.astype(str)), axis=1)

In [16]:
featured = featured .replace(',','', regex=True)

In [17]:
featured

0        Cowboy Bebop_Action Adventure Comedy Drama Sci...
1        Cowboy Bebop: Tengoku no Tobira_Action Drama M...
2        Trigun_Action Sci-Fi Adventure Comedy Drama Sh...
3        Witch Hunter Robin_Action Mystery Police Super...
4        Bouken Ou Beet_Adventure Fantasy Shounen Super...
                               ...                        
16209    Daomu Biji Zhi Qinling Shen Shu_Adventure Myst...
16210    Mieruko-chan_Comedy Horror Supernatural_Unknow...
16211    Higurashi no Naku Koro ni Sotsu_Mystery Dement...
16212    Yama no Susume: Next Summit_Adventure Slice of...
16213    Scarlet Nexus_Action Fantasy_Unknown_Sunrise_U...
Length: 16214, dtype: object

In [147]:
anime_description.rename(columns={"MAL_ID": "anime_id"}, inplace=True)

,MAL_ID,Name,Score,Genres,sypnopsis
0,1,Cowboy Bebop,8.78,"Action, Adventure, Comedy, Drama, Sci-Fi, Space","In the year 2071, humanity has colonized sever..."
1,5,Cowboy Bebop: Tengoku no Tobira,8.39,"Action, Drama, Mystery, Sci-Fi, Space","other day, another bounty—such is the life of ..."
2,6,Trigun,8.24,"Action, Sci-Fi, Adventure, Comedy, Drama, Shounen","Vash the Stampede is the man with a $$60,000,0..."
3,7,Witch Hunter Robin,7.27,"Action, Mystery, Police, Supernatural, Drama, ...",ches are individuals with special powers like ...
4,8,Bouken Ou Beet,6.98,"Adventure, Fantasy, Shounen, Supernatural",It is the dark century and the people are suff...
...,...,...,...,...,...
16209,48481,Daomu Biji Zhi Qinling Shen Shu,Unknown,"Adventure, Mystery, Supernatural",No synopsis information has been added to this...
16210,48483,Mieruko-chan,Unknown,"Comedy, Horror, Supernatural",ko is a typical high school student whose life...
16211,48488,Higurashi no Naku Koro ni Sotsu,Unknown,"Mystery, Dementia, Horror, Psychological, Supe...",Sequel to Higurashi no Naku Koro ni Gou .
16212,48491,Yama no Susume: Next Summit,Unknown,"Adventure, Slice of Life, Comedy",New Yama no Susume anime.


In [18]:
vectorizer = TfidfVectorizer()

In [19]:
feature_vectors = vectorizer.fit_transform(featured)

In [20]:
print(feature_vectors)

  (0, 11232)	0.03725364267326765
  (0, 48521)	0.044271829492614094
  (0, 36769)	0.030660488632878506
  (0, 47957)	0.045463325869260246
  (0, 35615)	0.10376615923725806
  (0, 64488)	0.0947534033421958
  (0, 34195)	0.09568642819989229
  (0, 19356)	0.07132644034983315
  (0, 73930)	0.07859394126746425
  (0, 65063)	0.05018313341078838
  (0, 33073)	0.07654365625315983
  (0, 42935)	0.05930743645276868
  (0, 10874)	0.06015152002821227
  (0, 22745)	0.11657222994739565
  (0, 33573)	0.04263447759466045
  (0, 73894)	0.05040065026843234
  (0, 43149)	0.09866657845681602
  (0, 52738)	0.05300673373455562
  (0, 48199)	0.04327725394235146
  (0, 22062)	0.05816531921030628
  (0, 45696)	0.060244033116192315
  (0, 24863)	0.04574103423462512
  (0, 72138)	0.07859394126746425
  (0, 11123)	0.05701905813989433
  (0, 69576)	0.08916321757797344
  :	:
  (16213, 54834)	0.1420230302097039
  (16213, 59599)	0.11225097845781959
  (16213, 25432)	0.11689387634909258
  (16213, 24537)	0.09828180253554712
  (16213, 53004)	0.

Cosine Similarity

In [21]:
similarity = cosine_similarity(feature_vectors)

In [22]:
similarity.shape

(16214, 16214)

Getting the Anime Name

In [23]:
anime_name = input('Enter anime name : ')

In [24]:
title_list = anime_full.Name.to_list()

In [25]:
find_close_match = difflib.get_close_matches(anime_name, title_list)
anime_match = find_close_match[0]


In [26]:
index_of_anime = anime_full[anime_full.Name == anime_match].index.values[0]

In [27]:
index_of_anime

10

In [28]:
similarity_score = list(enumerate(similarity[index_of_anime]))

In [29]:
most_similar_animes = sorted(similarity_score, key=lambda x:x[1], reverse=True)

In [30]:
print("Suggested Animes for your sunday night : \n")

for anime in most_similar_animes[:10]:
    print(anime_full[anime_full.index == anime[0]].Name.values[0])

Suggested Animes for your sunday night : 

Naruto
Naruto: Shippuuden
Naruto: Shippuuden Movie 6 - Road to Ninja
Boruto: Naruto Next Generations
Boruto: Naruto the Movie
Boruto: Naruto the Movie - Naruto ga Hokage ni Natta Hi
Naruto: Dai Katsugeki!! Yuki Hime Shinobu Houjou Dattebayo! - Konoha no Sato no Dai Undoukai
Naruto: Shippuuden Movie 4 - The Lost Tower
Naruto: Shippuuden Movie 5 - Blood Prison
The Last: Naruto the Movie


Actual Anime Recommendation Tool

In [31]:
anime_name = input('Enter anime name : ')
title_list = anime_full.Name.to_list()
find_close_match = difflib.get_close_matches(anime_name, title_list)
anime_match = find_close_match[0]
index_of_anime = anime_full[anime_full.Name == anime_match].index.values[0]
similarity_score = list(enumerate(similarity[index_of_anime]))
most_similar_animes = sorted(similarity_score, key=lambda x:x[1], reverse=True)
print("15 Best Suggested Animes for your sunday night : \n")
for anime in most_similar_animes[:15]:
    print(anime_full[anime_full.index == anime[0]].Name.values[0])


IndexError: list index out of range

In [ ]:
find_close_match

[]

## User Similarity Recommendation

In [35]:
user_short = user_review.head(1000000).sort_values(by=["user_id","anime_id"])

In [41]:
user_short.set_index("user_id")[["anime_id","rating"]]

,anime_id,rating
user_id,,
0,19,0
0,20,0
0,21,10
0,24,9
0,67,9
...,...,...
3404,40023,0
3404,40746,0
3404,40750,0


In [47]:
user_short

,user_id,anime_id,rating,watching_status,watched_episodes
65,0,19,0,6,0
33,0,20,0,2,220
4,0,21,10,1,0
5,0,24,9,1,5
0,0,67,9,1,1
...,...,...,...,...,...
999995,3404,40023,0,1,1
999996,3404,40746,0,1,5
999994,3404,40750,0,1,6
999998,3404,40801,4,2,2


In [53]:
Mean = user_short.groupby(by="user_id",as_index=False)['rating'].mean()
Rating_avg = pd.merge(user_short, Mean, on='user_id')
Rating_avg['adg_rating']=Rating_avg['rating_x']-Rating_avg['rating_y']
Rating_avg.head()

,user_id,anime_id,rating_x,watching_status,watched_episodes,rating_y,adg_rating
0,0,19,0,6,0,4.810811,-4.810811
1,0,20,0,2,220,4.810811,-4.810811
2,0,21,10,1,0,4.810811,5.189189
3,0,24,9,1,5,4.810811,4.189189
4,0,67,9,1,1,4.810811,4.189189


In [55]:
user_rating = pd.pivot_table(Rating_avg, values='adg_rating',index='user_id',columns='anime_id')

In [77]:
#Sparse Matrix
#Maybe use matrix factorization
user_rating

anime_id,1,5,6,7,8,15,16,17,18,19,...,48413,48417,48418,48426,48438,48441,48456,48471,48483,48488
user_id,,,,,,,,,,,,,,,,,,,,,
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-4.810811,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.223022,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,8.04251,NaN,NaN,NaN,-0.957490,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2.189944,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.189944,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,2.955128,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3400,5.107595,-4.892405,1.107595,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-4.892405,-4.892405,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3401,1.508475,NaN,-4.491525,NaN,NaN,NaN,NaN,NaN,NaN,3.508475,...,-4.491525,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3402,1.647849,-0.352151,0.647849,NaN,NaN,NaN,NaN,NaN,NaN,0.647849,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [78]:
user_rat = user_rating.fillna(user_rating.mean(axis=0))

anime_rat = user_rating.apply(lambda row: row.fillna(row.mean()), axis=1)


KeyboardInterrupt: 

In [79]:
user_rat

anime_id,1,5,6,7,8,15,16,17,18,19,...,48413,48417,48418,48426,48438,48441,48456,48471,48483,48488
user_id,,,,,,,,,,,,,,,,,,,,,
0,0.626245,1.446789,0.460985,-1.216742,-0.895606,-0.242675,-1.177943,0.313476,0.647573,-4.810811,...,-4.257506,-4.285864,-5.026473,-6.344874,-3.902138,-4.642857,-0.790945,-4.631068,-4.247159,-4.315245
1,0.626245,1.446789,0.460985,-1.216742,-0.895606,-0.242675,-1.177943,0.313476,0.647573,2.223022,...,-4.257506,-4.285864,-5.026473,-6.344874,-3.902138,-4.642857,-0.790945,-4.631068,-4.247159,-4.315245
2,0.626245,1.446789,0.460985,-1.216742,-0.895606,8.042510,-1.177943,0.313476,0.647573,-0.957490,...,-4.257506,-4.285864,-5.026473,-6.344874,-3.902138,-4.642857,-0.790945,-4.631068,-4.247159,-4.315245
3,2.189944,1.446789,0.460985,-1.216742,-0.895606,-0.242675,-1.177943,0.313476,1.189944,-0.978216,...,-4.257506,-4.285864,-5.026473,-6.344874,-3.902138,-4.642857,-0.790945,-4.631068,-4.247159,-4.315245
4,0.626245,1.446789,0.460985,-1.216742,-0.895606,-0.242675,2.955128,0.313476,0.647573,-0.978216,...,-4.257506,-4.285864,-5.026473,-6.344874,-3.902138,-4.642857,-0.790945,-4.631068,-4.247159,-4.315245
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3400,5.107595,-4.892405,1.107595,-1.216742,-0.895606,-0.242675,-1.177943,0.313476,0.647573,-0.978216,...,-4.892405,-4.892405,-5.026473,-6.344874,-3.902138,-4.642857,-0.790945,-4.631068,-4.247159,-4.315245
3401,1.508475,1.446789,-4.491525,-1.216742,-0.895606,-0.242675,-1.177943,0.313476,0.647573,3.508475,...,-4.491525,-4.285864,-5.026473,-6.344874,-3.902138,-4.642857,-0.790945,-4.631068,-4.247159,-4.315245
3402,1.647849,-0.352151,0.647849,-1.216742,-0.895606,-0.242675,-1.177943,0.313476,0.647573,0.647849,...,-4.257506,-4.285864,-5.026473,-6.344874,-3.902138,-4.642857,-0.790945,-4.631068,-4.247159,-4.315245


In [62]:
cosine = cosine_similarity(user_rat)
np.fill_diagonal(cosine,0)

In [137]:
similarity_with_anime= pd.DataFrame(cosine,index=user_rat.index)
similarity_with_anime

,0,1,2,3,4,5,6,7,8,9,...,3125,3126,3127,3128,3129,3130,3131,3132,3133,3134
user_id,,,,,,,,,,,,,,,,,,,,,
0,0.000000,0.972745,0.945474,0.963852,0.965827,0.957541,0.911488,0.960689,0.972983,0.985071,...,0.945745,0.937273,0.939515,0.975278,0.888167,0.905292,0.946965,0.976257,0.953108,0.984640
1,0.972745,0.000000,0.950595,0.964332,0.969194,0.960427,0.909924,0.965550,0.976005,0.988251,...,0.947914,0.938007,0.943894,0.977160,0.893654,0.909695,0.949536,0.981180,0.953863,0.986796
2,0.945474,0.950595,0.000000,0.934951,0.943800,0.933179,0.888098,0.938235,0.945625,0.961170,...,0.919221,0.913483,0.920934,0.946858,0.883405,0.884292,0.924730,0.950933,0.927590,0.960901
3,0.963852,0.964332,0.934951,0.000000,0.956683,0.943539,0.900707,0.952485,0.962304,0.974956,...,0.935402,0.928699,0.930105,0.966124,0.878644,0.899459,0.936790,0.967388,0.940092,0.974429
4,0.965827,0.969194,0.943800,0.956683,0.000000,0.952559,0.899767,0.952374,0.965101,0.979449,...,0.937863,0.931112,0.935427,0.968218,0.888577,0.902271,0.943597,0.971178,0.946818,0.979623
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3400,0.905292,0.909695,0.884292,0.899459,0.902271,0.891032,0.853837,0.898252,0.906980,0.920940,...,0.885111,0.876826,0.872729,0.910642,0.836616,0.000000,0.883278,0.917233,0.884783,0.920599
3401,0.946965,0.949536,0.924730,0.936790,0.943597,0.932860,0.886379,0.936150,0.947043,0.960482,...,0.919672,0.911651,0.918421,0.949968,0.865947,0.883278,0.000000,0.950797,0.926106,0.960644
3402,0.976257,0.981180,0.950933,0.967388,0.971178,0.962203,0.914786,0.966743,0.980715,0.991658,...,0.950768,0.941772,0.941313,0.982414,0.898341,0.917233,0.950797,0.000000,0.955694,0.990949


In [138]:
def find_n_neighbours(df,n):
    order = np.argsort(df.values, axis=1)[:, :n]
    df = df.apply(lambda x: pd.Series(x.sort_values(ascending=False)
           .iloc[:n].index, 
          index=['top{}'.format(i) for i in range(1, n+1)]), axis=1)
    return df

In [139]:
similarity_with_anime.iloc[:30].index

Int64Index([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
            17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29],
           dtype='int64', name='user_id')

In [140]:
sim_user_30_m = find_n_neighbours(similarity_with_anime,30)
sim_user_30_m.head()

,top1,top2,top3,top4,top5,top6,top7,top8,top9,top10,...,top21,top22,top23,top24,top25,top26,top27,top28,top29,top30
user_id,,,,,,,,,,,,,,,,,,,,,
0,3118,197,30,1114,2384,1912,163,2821,1977,2051,...,2730,2963,1020,2718,479,2104,2113,673,1554,2492
1,2682,443,3037,2593,1245,1864,1336,2750,2467,719,...,182,407,650,565,1032,2616,1178,1439,1501,1436
2,3063,3080,1003,251,2072,1864,1010,1439,2670,101,...,565,1032,2616,1178,402,2113,2197,719,1436,1669
3,2062,2124,2384,2474,1977,2963,197,1891,2113,607,...,1455,2793,1196,182,407,9,565,1032,2616,650
4,2062,2124,443,2384,1715,1162,2474,2113,878,2963,...,1975,1455,26,2793,182,650,565,2616,402,1501


In [141]:
user_rat.head()

anime_id,1,5,6,7,8,15,16,17,18,19,...,48413,48417,48418,48426,48438,48441,48456,48471,48483,48488
user_id,,,,,,,,,,,,,,,,,,,,,
0,0.626245,1.446789,0.460985,-1.216742,-0.895606,-0.242675,-1.177943,0.313476,0.647573,-4.810811,...,-4.257506,-4.285864,-5.026473,-6.344874,-3.902138,-4.642857,-0.790945,-4.631068,-4.247159,-4.315245
1,0.626245,1.446789,0.460985,-1.216742,-0.895606,-0.242675,-1.177943,0.313476,0.647573,2.223022,...,-4.257506,-4.285864,-5.026473,-6.344874,-3.902138,-4.642857,-0.790945,-4.631068,-4.247159,-4.315245
2,0.626245,1.446789,0.460985,-1.216742,-0.895606,8.042510,-1.177943,0.313476,0.647573,-0.957490,...,-4.257506,-4.285864,-5.026473,-6.344874,-3.902138,-4.642857,-0.790945,-4.631068,-4.247159,-4.315245
3,2.189944,1.446789,0.460985,-1.216742,-0.895606,-0.242675,-1.177943,0.313476,1.189944,-0.978216,...,-4.257506,-4.285864,-5.026473,-6.344874,-3.902138,-4.642857,-0.790945,-4.631068,-4.247159,-4.315245
4,0.626245,1.446789,0.460985,-1.216742,-0.895606,-0.242675,2.955128,0.313476,0.647573,-0.978216,...,-4.257506,-4.285864,-5.026473,-6.344874,-3.902138,-4.642857,-0.790945,-4.631068,-4.247159,-4.315245


In [142]:
user_rat.loc[:,5]

user_id
0       1.446789
1       1.446789
2       1.446789
3       1.446789
4       1.446789
          ...   
3400   -4.892405
3401    1.446789
3402   -0.352151
3403    2.762463
3404    1.446789
Name: 5, Length: 3135, dtype: float64

In [143]:
def User_item_score(user,item):
    #Get values from the 30 most similar users
    a = sim_user_30_m[sim_user_30_m.index==user].values
    b = a.squeeze().tolist()
    c = user_rat.loc[:,item]
    d = c[c.index.isin(b)]
    #Only the users with the same anime 
    f = d[d.notnull()]

    #Mean rating of --user
    avg_user = Mean.loc[Mean['user_id'] == user,'rating'].values[0]
    print(avg_user)
    index = f.index.values.squeeze().tolist()

    #Correlation 
    corr = similarity_with_anime.loc[user,index]


    fin = pd.concat([f, corr], axis=1)
    fin.columns = ['adg_score','correlation']
    fin['score']=fin.apply(lambda x:x['adg_score'] * x['correlation'],axis=1)
    nume = fin['score'].sum()
    deno = fin['correlation'].sum()
    final_score = avg_user + (nume/deno)
    return final_score

score = User_item_score(220,250)
print("score (u,i) is",score)

8.8
score (u,i) is 9.014305701918603


In [144]:
Rating_avg = Rating_avg.astype({"anime_id": str})
anime_user = Rating_avg.groupby(by = 'user_id')['anime_id'].apply(lambda x:','.join(x))

In [145]:
anime_user

user_id
0       19,20,21,24,67,68,71,121,134,164,169,174,177,1...
1       19,20,21,22,154,199,269,442,481,936,995,1535,1...
2       15,19,20,21,22,24,71,73,189,190,226,235,239,24...
3       1,18,25,30,43,47,57,60,66,71,73,101,131,134,18...
4       16,59,61,68,71,74,100,101,103,104,105,120,123,...
                              ...                        
3400    1,5,6,20,24,26,30,45,47,66,80,121,164,166,199,...
3401    1,6,19,30,32,47,164,199,205,226,227,228,263,26...
3402    1,5,6,19,20,21,30,32,43,44,45,47,67,109,114,12...
3403    1,5,26,30,31,32,47,54,71,72,73,81,82,83,84,85,...
3404              4181,9989,40023,40746,40750,40801,41226
Name: anime_id, Length: 3135, dtype: object

In [167]:
def User_item_score1(user):
    anime_seen_by_user = user_rating.columns[user_rating[user_rating.index==user].notna().any()].tolist()
    a = sim_user_30_m[sim_user_30_m.index==user].values
    b = a.squeeze().tolist()
    d = anime_user[anime_user.index.isin(b)]
    l = ','.join(d.values)
    anime_seen_by_similar_users = l.split(',')
    animes_under_consideration = list(set(anime_seen_by_similar_users)-set(list(map(str, anime_seen_by_user))))
    #print(animes_under_consideration)
    animes_under_consideration = list(map(int, animes_under_consideration))
    score = []
    for item in animes_under_consideration:
        c = user_rat.loc[:,item]
        d = c[c.index.isin(b)]
        f = d[d.notnull()]
        avg_user = Mean.loc[Mean['user_id'] == user,'rating'].values[0]
        index = f.index.values.squeeze().tolist()
        corr = similarity_with_anime.loc[user,index]
        fin = pd.concat([f, corr], axis=1)
        fin.columns = ['adg_score','correlation']
        fin['score']=fin.apply(lambda x:x['adg_score'] * x['correlation'],axis=1)
        nume = fin['score'].sum()
        deno = fin['correlation'].sum()
        final_score = avg_user + (nume/deno)
        score.append(final_score)
    data = pd.DataFrame({'anime_id':animes_under_consideration,'score':score})
    top_5_recommendation = data.sort_values(by='score',ascending=False).head(5)
    anime_Name = top_5_recommendation.merge(anime_description, how='inner', on='anime_id')
    anime_Names = anime_Name.Name.values.tolist()
    return anime_Names

In [168]:
anime_description

,anime_id,Name,Score,Genres,sypnopsis
0,1,Cowboy Bebop,8.78,"Action, Adventure, Comedy, Drama, Sci-Fi, Space","In the year 2071, humanity has colonized sever..."
1,5,Cowboy Bebop: Tengoku no Tobira,8.39,"Action, Drama, Mystery, Sci-Fi, Space","other day, another bounty—such is the life of ..."
2,6,Trigun,8.24,"Action, Sci-Fi, Adventure, Comedy, Drama, Shounen","Vash the Stampede is the man with a $$60,000,0..."
3,7,Witch Hunter Robin,7.27,"Action, Mystery, Police, Supernatural, Drama, ...",ches are individuals with special powers like ...
4,8,Bouken Ou Beet,6.98,"Adventure, Fantasy, Shounen, Supernatural",It is the dark century and the people are suff...
...,...,...,...,...,...
16209,48481,Daomu Biji Zhi Qinling Shen Shu,Unknown,"Adventure, Mystery, Supernatural",No synopsis information has been added to this...
16210,48483,Mieruko-chan,Unknown,"Comedy, Horror, Supernatural",ko is a typical high school student whose life...
16211,48488,Higurashi no Naku Koro ni Sotsu,Unknown,"Mystery, Dementia, Horror, Psychological, Supe...",Sequel to Higurashi no Naku Koro ni Gou .
16212,48491,Yama no Susume: Next Summit,Unknown,"Adventure, Slice of Life, Comedy",New Yama no Susume anime.


In [170]:
user = int(input("Enter the user id to whom you want to recommend : "))
predicted_movies = User_item_score1(user)
print(" ")
print("The Recommendations for User Id : ", user)
print("   ")
for i in predicted_movies:
    print(i)

 
The Recommendations for User Id :  102
   
Furiko
Sayonara Gokko
Fate/stay night: Unlimited Blade Works 2nd Season
Mahou Shoujo Madoka★Magica Movie 3: Hangyaku no Monogatari
High School DxD New


In [129]:
similarity_with_anime.loc[220, [1,2]]

1    0.988013
2    0.959944
Name: 220, dtype: float64